# Projet 2, TP1 - Régression sans a priori

Dans le projet précédent, on s'est basé sur une observation des données pour choisir un modèle (en l'occurrence, une représentation affine). 
Ici, on va encore considérer des données artificielles, mais la fonction que l'on cherche à représenter ne sera pas aussi évidente.
On parle alors de régression sans *a priori*, ce qui nous amènera rapidement aux [perceptrons multi-couches](https://www.youtube.com/watch?v=FBpPjjhJGhk&list=PLPz3O_BKtnTCoJlcRIrhtQ7CDewWhzI7x&index=1&pp=iAQB).

Ce premier TP se concentre sur le concept d'interpolation, qui consiste à obtenir une fonction continue qui colle parfaitement aux points de données, et qui permet d'estimer la fonction *entre* ces points.
La plus connue est l'interpolation linéaire, où on relie juste deux points de données consécutifs par une droite.
Ici, on s'intéresse à une méthode plus régulière (i.e. qui donne des fonctions moins «anguleuses»), appelée l'interpolation par splines cubiques.
Celle-ci est implémentée dans [SciPy](https://scipy.org/), et nous n'avons pas besoin de la comprendre en détail.

Après une première partie de génération de données, la deuxième partie s'intéresse aux résultats d'interpolation et d'extrapolation de cette méthode. 
On effectue d'abord une interpolations sur les données lisses (sans bruit), puis sur des données bruitées (comme au TP précédent).
En deuxième partie, on ajoute du bruit sur les données, comme pour le TP précédent.
On observe alors l'impact sur cette méthode d'interpolation.

In [64]:
## Les imports utiles

import numpy as np

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import qualitative as pc

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from utils import DEFAULT_SEED, torch_rng

from scipy.interpolate import make_interp_spline

## Partie 1 - Génération et affichage des données

### 1.1. La fonction sous-jacente

In [65]:
def f(x):
    b = 3.5
    a = 3.5 ** (np.log(0.5) / np.log(3.0))
    w = np.sin(2.0 * np.pi * x) + a * np.sin(2.0 * np.pi * b * x)
    return np.exp(w -((x + 0.1) ** 2))

x_min, x_max = -1.0, 0.9
x_range = x_max - x_min

# continuous data for plotting
x_cont = np.linspace(x_min, x_max, 800)
y_cont = f(x_cont)

continuous_trace = go.Scatter(x=x_cont, y=y_cont, name="underlying function", line=dict(color=pc.Plotly[0]))

### 1.2. Génération des données

In [66]:
rng_seed = sum(ord(c) for c in "R5.A.12-ModMath")
rng = np.random.default_rng(rng_seed)

n_data = 100

# points de collocation, inputs
x_data = rng.uniform(x_min + 0.05 * x_range, x_max - 0.05 * x_range, n_data)

# outputs non bruitées
y_data_no_noise = f(x_data)

# outputs bruitées
noise_amplitude = 0.25
noise_data = rng.normal(0.0, noise_amplitude, y_data_no_noise.shape)
y_data_noisy = y_data_no_noise + noise_data

### 1.3. Affichage des données

**TODO:**  
Afficher la fonction continue, ainsi que les points de donnée bruités et non bruités. 
Le graphique devra avoir une légende pertinente, grâce à l'argument `name` lors de la création des traces.

In [67]:
no_noise_params = dict(
    marker=dict(color=pc.Plotly[1], size=8),
    name="clean data points",
)
no_noise_scatter = go.Scatter(
    x=x_data, y=y_data_no_noise, mode="markers", **no_noise_params
)
noisy_params = dict(marker=dict(color=pc.Plotly[2], symbol="diamond", size=7), name="noisy data points")
noisy_scatter = go.Scatter(x=x_data, y=y_data_noisy, mode="markers", **noisy_params)

fig = go.Figure(data=[continuous_trace, no_noise_scatter, noisy_scatter])
fig.update_layout(width=800, height=350, margin=dict(l=20, r=20, b=20, t=20))
fig.show()

## Partie 2 - L'interpolation par splines cubiques

### 2.1. Interpolation sur données non bruitées

Est-ce que vous voyez immédiatement un défaut de cette méthode d'interpolation si on voulait, par exemple, l'appliquer à des images ?

In [68]:
idx_sort = np.argsort(x_data)
bspl_no_noise = make_interp_spline(x_data[idx_sort], y_data_no_noise[idx_sort])
# combien de paramètres ? 3 par intervalle, donc 3 * 99 = 297
no_noise_interp = go.Scatter(
    x=x_cont, y=bspl_no_noise(x_cont), name="clean spline interpolation"
)

fig = go.Figure(data=[continuous_trace, no_noise_interp, no_noise_scatter])

fig.update_layout(
    width=800, height=350, margin=dict(l=20, r=20, b=20, t=20)  # , showlegend=False
)
fig.update_yaxes(range=[-0.3, 3.3])

fig.show()

### 2.2. Interpolation bruitée

In [69]:
idx_sort = np.argsort(x_data)
bspl_noisy = make_interp_spline(x_data[idx_sort], y_data_noisy[idx_sort])
# combien de paramètres ? 3 par intervalle, donc 3 * 99 = 297
noisy_interp = go.Scatter(
    x=x_cont, y=bspl_noisy(x_cont), name="noisy spline interpolation", line=dict(color=pc.Plotly[2])
)

fig = go.Figure(data=[continuous_trace, noisy_interp, noisy_scatter])

fig.update_layout(
    width=800, height=350, margin=dict(l=20, r=20, b=20, t=20)  # , showlegend=False
)
fig.update_yaxes(range=[-0.3, 3.3])

fig.show()

## Partie 3 - Régression par perceptron multi-couches

Clairement, il faut changer d'approche, et privilégier une **régression**. Lors du TP précédent, on a étudié la régression linéaire en choisissant une représentation de fonction affine. Ici, on n'a aucune idée de quel genre de fonction choisir... Donc on va choisir un réseau de neurones ! L'architecture (i.e. la structure) la plus connue est le [perceptron multi-couches](https://www.youtube.com/watch?v=FBpPjjhJGhk&list=PLPz3O_BKtnTCoJlcRIrhtQ7CDewWhzI7x&index=1&pp=iAQB).

Combien de paramètres contient ce modèle ?

*Remarque :* Il y a aussi des méthodes statistiques de **débruitage** qui permettent un meilleur comportement des interpolations, mais on va se concentrer sur les MLP.

In [70]:
torch.manual_seed(DEFAULT_SEED)


class NeuronLayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, activation):
        super().__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)
        self.activation = activation()

    def forward(self, inputs):
        return self.activation(self.linear(inputs))


class MultiLayerPerceptron(nn.Module):
    def __init__(
        self, 
        in_dim=1, 
        out_dim=1, 
        hidden_dims=[20] * 4, 
        activation=nn.ReLU
    ):
        super().__init__()
        in_dims = [in_dim] + hidden_dims
        self.layers = nn.ModuleList() # []
        for d0, d1 in zip(in_dims, hidden_dims):
            self.layers.append(NeuronLayer(d0, d1, activation))
        self.layers.append(nn.Linear(in_dims[-1], out_dim))

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer(inputs)
        return inputs


mlp_relu = MultiLayerPerceptron(activation=nn.ReLU)
mlp_tanh = MultiLayerPerceptron(activation=nn.Tanh)
mlp_softplus = MultiLayerPerceptron(activation=nn.Softplus)

In [71]:
x_tensor = torch.tensor(x_data[:, None], dtype=torch.float32)
y_tensor = torch.tensor(y_data_noisy[:, None], dtype=torch.float32)
dataset = TensorDataset(x_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=len(dataset), generator=torch_rng())

In [72]:
def train(model, n_epochs, optim_algo=torch.optim.Adam, optim_params={"lr": 1e-3}):
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9995)

    for epoch in range(1_000):
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

train(mlp_relu, 4000)
train(mlp_tanh, 4000)
train(mlp_softplus, 4000)

In [73]:
fig = go.Figure()
fig.add_trace(continuous_trace)
x_cont_torch = torch.tensor(x_cont, dtype=torch.float)[:, None]
y_cont_relu = mlp_relu(x_cont_torch).detach().numpy()[:, 0]
y_cont_tanh = mlp_tanh(x_cont_torch).detach().numpy()[:, 0]
y_cont_softplus = mlp_softplus(x_cont_torch).detach().numpy()[:, 0]
fig.add_scatter(x=x_cont, y=y_cont_relu, name="MLP ReLU")
fig.add_scatter(x=x_cont, y=y_cont_tanh, name="MLP Tanh")
fig.add_scatter(x=x_cont, y=y_cont_softplus, name="MLP Softplus")
fig.add_scatter(x=x_data, y=y_data_noisy, mode="markers", name="data points")

fig.update_layout(
    width=800, height=350, margin=dict(l=20, r=20, b=20, t=20)
)
fig.update_yaxes(range=[-0.3, 3.3])

fig.show()